In [14]:
import pandas as pd
import numpy as np
import random
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils
np.random.seed(1)
random.seed(1)

TRAININFO = "/media/DATA/zalo-hit-song-prediction/csv/train_info.tsv"
TRAINRANK =  "/media/DATA/zalo-hit-song-prediction/csv/train_rank.csv"
TESTINFO = "/media/DATA/zalo-hit-song-prediction/csv/test_info.tsv"
Track_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_info.csv"
Audio_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_audio_features.csv"
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()

with open("/media/DATA/zalo-hit-song-prediction/csv/stopwords/vietnamese-stopwords.txt", 'r') as f:
    filecontent=f.readlines()
stopwords = list(set([f.strip() for f in filecontent]))


stopwords_tokenized = list(set([ViTokenizer.tokenize(stopword.lower()) for stopword in stopwords]))
set([ViTokenizer.tokenize(word) for word in stopwords_tokenized]) - set(stopwords_tokenized)
stopwords_tokenized[0]
df_train = df_train.drop_duplicates(subset='lyric', keep='first')

'bên bị'

In [72]:
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import HashingVectorizer

'''
def encode_column(df, column_name, to_n_columns=12):
    
    hasher=category_encoders.hashing.HashingEncoder()
    #encoded=hasher.fit_transform(np.expand_dims(df["university_name"].values, axis=-1),verbose=True, cols=None, drop_invariant=True, return_df=True, hash_method='sha1')
    tmp=pd.DataFrame(np.expand_dims(df["university_name"].values, axis=-1))
    encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)
    
    encoded_col_names=list(encoded_df.columns.values)
    tmp=df.copy()
    tmp["index"]=tmp.index
    tmp=tmp.reset_index()
    df=pd.merge(tmp, encoded_df, how='left', left_index=True, right_index=True, sort=False,
             suffixes=('_left', '_right'),validate="one_to_one")
    
    print(type(encoded_col_names))
    df.set_index("index",inplace=True)
    return df, encoded_col_names
'''
class TextColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


# generate features - in this case just one.
feature = Pipeline([
                ('selector_1', TextColumnSelector(key='lyric')),
                ('vect', HashingVectorizer(n_features = 2**4, 
                                           tokenizer=ViTokenizer.tokenize, 
                                           lowercase=True, 
                                           stop_words=stopwords_tokenized,
                                           norm='l2')),
                ]
                #('vect', CountVectorizer(analyzer='word', tokenizer=str.split, stop_words=stopwords)),
                #('tfidf', TfidfTransformer(use_idf=True, sublinear_tf=True))
                )
# Another option:
##hasher=category_encoders.hashing.HashingEncoder()
##encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)
#vectorizer = HashingVectorizer(n_features = 2**4, tokenizer=word_tokenize, lowercase=True, stop_words=stopwords,  norm='l2')
#vectors = vectorizer.fit_transform(df_train.lyric)

feats_list=[('text', feature)]
feats = FeatureUnion(feats_list)


'từ giờ'

In [93]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
def rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))
rmse_scorer = make_scorer(rmse, greater_is_better=False)


#model=LogisticRegression(class_weight="balanced",penalty='l1', random_state=1)
#model_lc="classifier__C"
#values=[719.686, ] # np.logspace(-4, 4, 8) # 0.001 to 1e4
#http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html 
model=SVR(kernel="rbf", C=np.logspace(-4, 4, 8))
model_lc="classifier__gamma"
values = np.logspace(-4, 4, 20)
model_lc_2 = "classifier__C"
values2 = np.logspace(-4, 4, 20)


model_pipeline = Pipeline([
    ('features',feats),
    ('classifier', model),
])

#param_range = np.logspace(1, 10, 5)

parameters = {'features__text__vect__n_features': np.array([0.5e3, 1e3,1.5e3]).astype('int64'),
              'features__text__vect__ngram_range': [(1,1), (1,2)],# [(1,1), (1, 2),], # try bigrams or unigrams
              model_lc: values,
              model_lc_2:values2,
             }

skf = StratifiedKFold(n_splits=10, random_state=99999)
gs_clf = RandomizedSearchCV(model_pipeline, 
                      parameters, 
                      scoring=rmse_scorer, 
                      cv=skf,
                      n_jobs=-1,
                      return_train_score=False,
                      error_score='raise',
                      n_iter=100,
                      verbose=10
                    )

#gs_clf.get_params().keys()
#[i for i in gs_clf.get_params().keys()]
gs_clf.fit(df_train, df_train.label)

print(gs_clf.cv_results_['mean_test_score'][gs_clf.best_index_], "+/-" ,gs_clf.cv_results_['std_test_score'][gs_clf.best_index_])
print(gs_clf.best_score_, gs_clf.best_params_)
pd.DataFrame(gs_clf.cv_results_).to_csv('svr_lyric_bigram2.csv')

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 10

-2.9067750914614843 +/- 0.14068620167340345
-2.9067750914614843 {'features__text__vect__ngram_range': (1, 2), 'features__text__vect__n_features': 1500, 'classifier__gamma': 11.288378916846883, 'classifier__C': 0.615848211066026}


In [100]:
from sklearn.externals import joblib
joblib.dump(gs_clf.best_estimator_, 'svr_lyric_bigram2.pkl')

['svr_lyric_bigram2.pkl']

In [101]:
df_train.to_csv('svr_lyric_bigram2_df.csv')

In [98]:
pd.DataFrame(gs_clf.cv_results_)[['param_classifier__C','param_classifier__gamma', 'param_features__text__vect__n_features', 'param_features__text__vect__n_features', 'param_features__text__vect__ngram_range','mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False)

,param_classifier__C,param_classifier__gamma,param_features__text__vect__n_features,param_features__text__vect__n_features,param_features__text__vect__ngram_range,mean_test_score,std_test_score
84,0.615848,11.2884,1500,1500,"(1, 2)",-2.906775,0.140686
74,3792.69,0.000695193,1500,1500,"(1, 2)",-2.917614,0.111916
42,0.615848,11.2884,1500,1500,"(1, 1)",-2.922199,0.108997
12,545.559,0.00483293,1000,1000,"(1, 2)",-2.932866,0.110951
25,1.62378,1.62378,1500,1500,"(1, 1)",-2.939385,0.074137
64,3792.69,0.0001,1000,1000,"(1, 2)",-2.940274,0.057017
27,1.62378,1.62378,1000,1000,"(1, 1)",-2.942360,0.074156
51,1.62378,1.62378,500,500,"(1, 1)",-2.943033,0.074382
98,545.559,0.00483293,1500,1500,"(1, 1)",-2.955742,0.073478
70,206.914,0.00483293,1500,1500,"(1, 1)",-2.961282,0.055085


In [52]:
import sklearn
def rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

dummy_clf = sklearn.dummy.DummyClassifier(strategy='stratified', random_state=99999)
skf = StratifiedKFold(n_splits=10, random_state=99999)
from sklearn.model_selection import cross_validate
cv_pred = cross_validate(dummy_clf, df_train['lyric'], df_train.label, scoring=rmse_scorer, cv=skf)
cv_pred['test_score'].mean(),  '+/-', cv_pred['test_score'].std()

(4.277847802266441, '+/-', 0.08664532999630303)

In [32]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)

array([1.e+09, 1.e+10])

In [115]:
import pandas as pd
import numpy as np
import random
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils
from nltk.tokenize import word_tokenize
import nltk
from sklearn.externals import joblib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import HashingVectorizer

def rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))
np.random.seed(1)
random.seed(1)

TRAININFO = "/media/DATA/zalo-hit-song-prediction/csv/train_info.tsv"
TRAINRANK =  "/media/DATA/zalo-hit-song-prediction/csv/train_rank.csv"
TESTINFO = "/media/DATA/zalo-hit-song-prediction/csv/test_info.tsv"
Track_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_info.csv"
Audio_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_audio_features.csv"
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()
'''
def encode_column(df, column_name, to_n_columns=12):

    hasher=category_encoders.hashing.HashingEncoder()
    #encoded=hasher.fit_transform(np.expand_dims(df["university_name"].values, axis=-1),verbose=True, cols=None, drop_invariant=True, return_df=True, hash_method='sha1')
    tmp=pd.DataFrame(np.expand_dims(df["university_name"].values, axis=-1))
    encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)

    encoded_col_names=list(encoded_df.columns.values)
    tmp=df.copy()
    tmp["index"]=tmp.index
    tmp=tmp.reset_index()
    df=pd.merge(tmp, encoded_df, how='left', left_index=True, right_index=True, sort=False,
             suffixes=('_left', '_right'),validate="one_to_one")

    print(type(encoded_col_names))
    df.set_index("index",inplace=True)
    return df, encoded_col_names
'''

class TextColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

with open("/media/DATA/zalo-hit-song-prediction/csv/stopwords/vietnamese-stopwords.txt", 'r') as f:
    filecontent=f.readlines()
stopwords = list(set([f.strip() for f in filecontent]))
stopwords_tokenized = list(set([ViTokenizer.tokenize(stopword.lower()) for stopword in stopwords]))
set([ViTokenizer.tokenize(word) for word in stopwords_tokenized]) - set(stopwords_tokenized)
df.dropna(subset=['lyric'], inplace=True) # drop songs with no lyrics
df_train = df[df.dataset=='train']
df_test = df[df.dataset=='test']
df_train = df_train.drop_duplicates(subset='lyric', keep='first')

# generate features - in this case just one.
feature = Pipeline([
    ('selector_1', TextColumnSelector(key='lyric')),
    ('vect', HashingVectorizer(n_features=1500,
                               tokenizer=ViTokenizer.tokenize,
                               lowercase=True,
                               stop_words=stopwords_tokenized,
                               norm='l2')),
]
    # ('vect', CountVectorizer(analyzer='word', tokenizer=str.split, stop_words=stopwords)),
    # ('tfidf', TfidfTransformer(use_idf=True, sublinear_tf=True))
)
# Another option:
##hasher=category_encoders.hashing.HashingEncoder()
##encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)
# vectorizer = HashingVectorizer(n_features = 2**4, tokenizer=word_tokenize, lowercase=True, stop_words=stopwords,  norm='l2')
# vectors = vectorizer.fit_transform(df_train.lyric)

feats_list = [('text', feature)]
feats = FeatureUnion(feats_list)
rmse_scorer = make_scorer(rmse, greater_is_better=False)
#model=LogisticRegression(class_weight="balanced",penalty='l1', random_state=1)
#model_lc="classifier__C"
#values=[719.686, ] # np.logspace(-4, 4, 8) # 0.001 to 1e4
model2=SVR(kernel="rbf", C=0.615848, gamma=11.2884)

model_pipeline = Pipeline([
    ('features',feats),
    ('classifier', model2),
])
'''
model_lc="classifier__gamma"
values = [11.2884,]
model_lc_2 = "classifier__C"
values2 = [0.615848,]

model_pipeline = Pipeline([
    ('features',feats),
    ('classifier', model),
])

#param_range = np.logspace(1, 10, 5)

parameters = {'features__text__vect__n_features': np.array([1500,]).astype('int64'),
              'features__text__vect__ngram_range': [(1,2),],# [(1,1), (1, 2),], # try bigrams or unigrams
              model_lc: values,
              model_lc_2:values2,
             }

skf = StratifiedKFold(n_splits=10, random_state=99999)
gs_clf = RandomizedSearchCV(model_pipeline,
                      parameters,
                      scoring=rmse_scorer,
                      cv=skf,
                      n_jobs=-1,
                      return_train_score=False,
                      error_score='raise',
                      n_iter=100,
                      verbose=10
                    )

#gs_clf.get_params().keys()
#[i for i in gs_clf.get_params().keys()]
gs_clf.fit(df_train, df_train.label)

print(gs_clf.cv_results_['mean_test_score'][gs_clf.best_index_], "+/-" ,gs_clf.cv_results_['std_test_score'][gs_clf.best_index_])
print(gs_clf.best_score_, gs_clf.best_params_)
pd.DataFrame(gs_clf.cv_results_).to_csv('svr_lyric_bigram2.csv')
'''
model_pipeline.fit(df_train, df_train.label)
joblib.dump(model2, 'svr_lyric_bigram2_trained_on_whole_set.pkl')
preds = model_pipeline.predict(df_train)
df_train['lyric_feature'] = preds
df_train[['ID', 'lyric_feature']].to_csv('../csv/lyric_feature.csv')

['svr_lyric_bigram2_trained_on_whole_set.pkl']

FileNotFoundError: [Errno 2] No such file or directory: '../csv/lyric_feature.csv'

In [118]:
orig_preds = gs_clf.best_estimator_.predict(df_train)

In [119]:
[(i,j) for i, j in zip(preds, orig_preds)]

[(6.823672486298127, 6.899990896979058),
 (5.392184965042551, 5.910539394477836),
 (6.364384036837183, 6.3199401291603525),
 (5.555291686776725, 5.873810944332039),
 (6.432250564483906, 6.405219116233078),
 (7.231975309537795, 6.899234005798),
 (6.223658710601299, 5.707991607454733),
 (6.131528428477795, 5.035589148984181),
 (5.619035303666289, 5.695902792114748),
 (4.92663762940926, 4.900152551298229),
 (6.415380007322631, 6.806716567153392),
 (5.088319720005189, 5.7290425788091985),
 (6.006137577307461, 5.833608726075657),
 (5.457830896878757, 5.231753455473917),
 (6.072631351768861, 5.3488186069529915),
 (7.2044385512396865, 7.122033870846289),
 (7.794222235915169, 7.505240075788304),
 (4.0065846933679055, 3.125543175773757),
 (6.4617857185694865, 6.188158401752387),
 (7.343379855288959, 7.284197643053683),
 (4.129192268820665, 4.8483040584578605),
 (6.354166223404491, 6.703845533878999),
 (6.27331868485512, 5.871161027939828),
 (5.916317856061465, 6.128881601904177),
 (5.8583771915

In [35]:


import pandas as pd
import numpy as np
import random
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils
np.random.seed(1)
random.seed(1)

TRAININFO = "/media/DATA/zalo-hit-song-prediction/csv/train_info.tsv"
TRAINRANK =  "/media/DATA/zalo-hit-song-prediction/csv/train_rank.csv"
TESTINFO = "/media/DATA/zalo-hit-song-prediction/csv/test_info.tsv"
Track_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_info.csv"
Audio_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_audio_features.csv"
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()

import joblib
joblib.dump(model_pipeline, 'svr_lyric_bigram2_trained_on_whole_set.pkl')
tmp = joblib.load('svr_lyric_bigram2_trained_on_whole_set.pkl')
df.dropna(subset=['lyric'], inplace=True) 
df_pred = tmp.predict(df)
df['lyric_feature'] = df_pred
df[['ID', 'lyric_feature']].to_csv('../csv/lyric_feature.csv')

In [39]:
df

,ID,title,artist_name,lyric,label,dataset,lyric_feature
0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,Đêm nay anh gánh dầu ra biển anh chôn \r\nAnh ...,7.0,train,6.823672
1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,Chiều mưa không có em\r\nbờ đá công viên âm th...,3.0,train,5.392185
2,1073835561,Rồi Ánh Trăng Tan,Lưu Bích,Rồi ánh trăng cũng đang tan dần\r\nRồi ước mơ ...,6.0,train,6.364384
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,Như Quỳnh,Nắng hạ đi Mây trôi lang thang cho hạ buồn Coi...,2.0,train,5.555292
4,1073929630,Người Điên Biết Yêu,Như Loan,"Ai trong tình yêu, ai không mơ mộng?\n\nNgu ng...",7.0,train,6.432251
5,1073929880,Đàn Bà,Don Hồ,Đã từ lâu tôi vẫn thường trong bóng đêm\r\nMan...,3.0,train,7.231975
6,1073949310,Yêu Một Người Sống Bên Một Người,Minh Tuyết,Những đêm buồn nghe mưa ngoài hiên mưa triền m...,4.0,train,6.223659
8,1073994298,Những Ân Tình Xưa,"Minh Tuyết, Bằng Kiều",Em có biết từ khi em ra đi trong lòng anh cả 1...,4.0,train,6.131528
9,1073994300,Ở Nơi Đó Em Cười,Minh Tuyết,"Một người buồn lắm, đêm nằm mơ người xưa ghé t...",8.0,train,5.619035
10,1074011942,Tình Chấp Nhận,Quỳnh Vi,Còn bao nhiêu giấc mơ ta vẫn mơ về\r\nCòn bao ...,5.0,train,4.926638
